# Creating a sample star schema in snowflake warehouse

## Using snowflake connector to perform database operations

In [15]:
import snowflake.connector as cnn
import pandas as pd

In [7]:
with open('config.txt', 'r') as conf:
    details = conf.readlines()

conn = cnn.connect(
    user=details[0],
    password=details[1],
    account=details[2],
    warehouse=details[3],
    database=details[4]
)

def create_database(conn, database_name):
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS {}".format(database_name))
    cursor.close()

create_database(conn, 'star_product_database')


In [8]:

def create_dimension_tables(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE OR REPLACE TABLE dim_product (
            product_id INTEGER PRIMARY KEY,
            product_name VARCHAR(100),
            category VARCHAR(50)
        )
    """)
    
    cursor.execute("""
        CREATE OR REPLACE TABLE dim_time (
            time_id TIMESTAMP PRIMARY KEY,
            year INTEGER,
            month INTEGER,
            day INTEGER
        )
    """)
    
    cursor.close()


def create_fact_table(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE OR REPLACE TABLE fact_sales (
            sales_id INTEGER PRIMARY KEY,
            product_id INTEGER,
            time_id TIMESTAMP,
            amount DECIMAL(18, 2),
            quantity INTEGER,
            FOREIGN KEY (product_id) REFERENCES dim_product(product_id),
            FOREIGN KEY (time_id) REFERENCES dim_time(time_id)
        )
    """)
    cursor.close()


create_dimension_tables(conn)
create_fact_table(conn)


In [9]:
def insert_dim_product_data(conn):
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO dim_product (product_id, product_name, category)
        VALUES
            (1, 'Product A', 'Category 1'),
            (2, 'Product B', 'Category 2')
    """)
    cursor.close()

def insert_dim_time_data(conn):
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO dim_time (time_id, year, month, day)
        VALUES
            ('2024-05-07 00:00:00', 2024, 5, 7)
    """)
    cursor.close()

def insert_fact_sales_data(conn):
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO fact_sales (sales_id, product_id, time_id, amount, quantity)
        VALUES
            (1, 1, '2024-05-07 00:00:00', 100.00, 5),
            (2, 2, '2024-05-07 00:00:00', 200.00, 3)
    """)
    cursor.close()

insert_dim_product_data(conn)
insert_dim_time_data(conn)
insert_fact_sales_data(conn)


In [41]:
def query_data(conn):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 
            p.product_id,
            f.time_id,
            f.quantity,
            f.amount
        FROM 
            fact_sales f
        JOIN 
            dim_product p ON f.product_id = p.product_id
    """)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    cursor.close()

def dim_table(conn):
    cursor = conn.cursor()
    cursor.execute(
    """
    select * from dim_product
    """
    )
    res = cursor.fetchall()
    for r in res:
        print(r)

query_data(conn)
print('/////')
dim_table(conn)

(1, datetime.datetime(2024, 5, 7, 0, 0), 5, Decimal('100.00'))
(2, datetime.datetime(2024, 5, 7, 0, 0), 3, Decimal('200.00'))
/////
(1, 'Product A', 'Category 1')
(2, 'Product B', 'Category 2')
(10, 'Cheese', 'breakfast')
(3, 'bread', 'breakfast')
(4, 'honey', 'breakfast')


## Using data from external API to enter into our star schema

In [22]:
# Function to retrieve data from the external API

def retrieve_data_from_api():
    
    response = requests.get('https://www.boredapi.com/api/activity')
    
    if response.status_code == 200:
        
        data = response.json()
        return data
    else:
        print('Error: Unable to retrieve data from the API')
        return None

def insert_data_into_database(data):
    if data:
        cursor = conn.cursor()
        for item in data:
            
            product_id = item['product_id']
            product_name = item['product_name']
            category = item['category']
            
            cursor.execute("""
                INSERT INTO dim_product (product_id, product_name, category)
                VALUES (%s, %s, %s)
            """, (product_id, product_name, category))
        cursor.close()
        conn.commit()
        print('Data inserted successfully into the database')
    else:
        print('No data to insert')

# api_data = retrieve_data_from_api()
# insert_data_into_database(api_data)

data = retrieve_data_from_api()
data

{'activity': 'Start a band',
 'type': 'music',
 'participants': 4,
 'price': 0.3,
 'link': '',
 'key': '5675880',
 'accessibility': 0.8}

When dealing with millions of records, it's more efficient to use bulk insert operations rather than executing individual `INSERT` statements for each record. Snowflake supports bulk insert operations, which can significantly improve the performance of data insertion.

One efficient way to insert large amounts of data into Snowflake is to use the `COPY INTO` command. This command allows you to bulk load data from various file formats (e.g., CSV, JSON, Parquet) stored in a cloud storage service (e.g., Amazon S3, Google Cloud Storage, Azure Blob Storage) directly into Snowflake tables.

Here's a general approach to bulk insert data into Snowflake using the `COPY INTO` command:

1. **Export Data to a File**: Export the data from your external API to a file in a supported format (e.g., CSV, JSON).

2. **Upload File to Cloud Storage**: Upload the file containing the data to a cloud storage service like Amazon S3, Google Cloud Storage, or Azure Blob Storage. Make sure the file is accessible to Snowflake.

3. **Execute COPY INTO Command**: Use the `COPY INTO` command in Snowflake to load the data from the file into your Snowflake table.


In [37]:
#file_location = 's3://bucket_name/path_to_file/data.csv'
file ='@~/sample_file.csv'

cursor = conn.cursor()
cursor.execute("""
    COPY INTO dim_product
    FROM '{}'          
    FILE_FORMAT = (TYPE = CSV)
""".format(file))
conn.commit()
cursor.close()


True

- We define the `file_location` variable, which contains the location of the file containing the data in cloud storage (e.g., Amazon S3).
- We execute the `COPY INTO` command to load the data from the file into the `dim_product` table in Snowflake. 

By using the `COPY INTO` command with bulk insert operations, you can efficiently load millions of records into your Snowflake tables without incurring the overhead of executing individual `INSERT` statements.

In [42]:
conn.close()